In [32]:
#Tools from Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
#Chrome setup 
CHROMEDRIVER_PATH = "/Users/(USER_NAME)/tools/chromedriver"
# Setup Chrome with headless mode OFF 
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Delete # to run invisibly
driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=options)
#Login
login_url = "https://cms.amptab.com/"
driver.get(login_url)
time.sleep(3)  # Give time for page to load

email = "email"
password = "password"

# Fill in login form
driver.find_element(By.NAME, "signinEmail").send_keys(email)
driver.find_element(By.NAME, "Password").send_keys(password)

# Click the login button
driver.find_element(By.ID, "signin_button_id").click()
time.sleep(3)  # wait for login to complete

# Click the "Shop" button after login
shop_button = driver.find_element(By.XPATH, '//a[contains(@href, "/Manufacturer/244482/Shop2Catalog")]')
shop_button.click()

# Wait for page to load
time.sleep(3)

#Find Living rooms category
category_button = driver.find_element(By.XPATH, '//a[@class= "filter-title-name collapsed" and @href="#collpase-0"]')
category_button.click()

time.sleep(2)

Living_rooms_button =  driver.find_element(By.XPATH, '//input[@type= "checkbox" and @name="filter_Category" and @value="LIVING+ROOM"]')
Living_rooms_button.click()

time.sleep(2)

submit_button = driver.find_element(By.XPATH, '//input[@id="search_button_id"]')
submit_button.click()

time.sleep(3)

#Change to list Format
list_button= driver.find_element(By.XPATH,'//i[@class="fal fa-list-alt fa-lg" and @aria-hidden="true"]')
list_button.click()


In [29]:
#For Loop to scrape for the amount of pages
import re
count_element= driver.find_element(By.XPATH, '//div[contains(text(), "Showing")]')
count_text = count_element.text.strip()

match=re.search(r" of (\d+)", count_text)
total_products = int(match.group(1) if match else 0)

items_per_page = 48
total_pages = (total_products // items_per_page) + (1 if total_products % items_per_page != 0 else 0)

#print(f"Total products: {total_products}")
#print(f"Total pages: {total_pages}")

In [30]:
all_data = []

for i in range(total_pages):
    tbodies = driver.find_elements(By.XPATH, '//tbody[starts-with(@id, "id-pn-")]')
    
    for tbody in tbodies:
        rows = tbody.find_elements(By.TAG_NAME, 'tr')

        for row in rows:
            try:
                tds = row.find_elements(By.TAG_NAME, 'td')
                
                # Only process rows with full data
                if len(tds) == 7:
                    # Use -3, -2, -1 only if you're SURE you're getting SKU, stock, price at the end
                    sku = tds[3].text.strip()
                    stock = tds[5].text.strip()
                    #price = tds[6].text.strip()
                elif len(tds) == 6:
                    # Use -3, -2, -1 only if you're SURE you're getting SKU, stock, price at the end
                    sku = tds[2].text.strip()
                    stock = tds[4].text.strip()
                    #price = tds[5].text.strip()
                else:
                    continue
                
                all_data.append({
                        'SKU': sku,
                        'Stock': stock,
                        #'Unit Price': price
                })
                
            except Exception as e:
                print("Row error:", e)
                continue

    try:
        next_button = driver.find_element(By.XPATH, '//a[text()=">"]')
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)
    except Exception as e:
        print("No more pages or navigation failed.")
        break     

In [31]:
import pandas as pd
df = pd.DataFrame(all_data)         
df.to_csv("raw_Inventory.csv", index=False) 
print("Saved Inventory ✅")

Saved Inventory ✅
